In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join('..', '..')))

import argparse
from pprint import pp
import torch
from torch import nn
from tqdm import tqdm
import numpy as np
import json
import os
from omegaconf import OmegaConf
from torch.utils.tensorboard import SummaryWriter

from lpn.utils import load_dataset, load_config
from lpn.utils import get_model
from lpn.utils import get_loss_hparams_and_lr, get_loss
from lpn.utils import trainer
from lpn.utils import utils
import matplotlib.pyplot as plt

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataset_config_path = "./configs/test_dataset.json"

In [ ]:
model_config_path = "./models/ne_lpn_sub_sq/s=0.1/model_config.json"
model_weight_path = "./models/ne_lpn_sub_sq/s=0.1/model.pt"

#import model as cuda
model_config = load_config(model_config_path)
model = get_model(model_config).to(device)
model.load_state_dict(torch.load(model_weight_path)["model_state_dict"])

init weights


<All keys matched successfully>

In [4]:
Lpn_config_path = "./models/lpn/s=0.1/model_config.json"
Lpn_weight_path = "./models/lpn/s=0.1/model.pt"

lpn_config = load_config(Lpn_config_path)
lpn = get_model(lpn_config).to(device)
lpn.load_state_dict(torch.load(Lpn_weight_path)["model_state_dict"])

init weights


<All keys matched successfully>

In [5]:
dataset_config = load_config(dataset_config_path)
test_dataset = load_dataset(dataset_config, "test")
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=4)

dataset:  celeba


In [6]:
scale_factors = [1, 2, 3, 4, 5]


for scale_factor in scale_factors:

    ne_running_loss = 0 
    lpn_running_loss = 0
    num_steps = 500
    for step, batch in enumerate(test_data_loader):
        if step >= num_steps:
            break

        clean_image = batch["image"].to(device)
        scaled_image = clean_image * scale_factor

        ne_clean_out = model(clean_image)
        lpn_clean_out = lpn(clean_image)

        ne_scaled_out = model(scaled_image)
        lpn_scaled_out = lpn(scaled_image)

        #shifted versions of clean outputs
        ne_clean_out_scaled = ne_clean_out * scale_factor
        lpn_clean_out_scaled = lpn_clean_out * scale_factor

        #loss between output from shifted input vs shifted output from clean input
        mse_loss = nn.MSELoss()
        ne_loss = mse_loss(ne_scaled_out, ne_clean_out_scaled)
        lpn_loss = mse_loss(lpn_scaled_out, lpn_clean_out_scaled)

        ne_running_loss += ne_loss.item()
        lpn_running_loss += lpn_loss.item()
    
    ne_avg_loss = ne_running_loss / num_steps
    lpn_avg_loss = lpn_running_loss / num_steps

    print(f"Scale factor: {scale_factor}")
    print(f"NE avg loss: {ne_avg_loss}")
    print(f"LPN avg loss: {lpn_avg_loss}")
    print(f"LPN avg loss - NE avg loss: {lpn_avg_loss - ne_avg_loss}\n")

Scale factor: 1
NE avg loss: 3.5214656283849325e-16
LPN avg loss: 2.1363409285040197e-15
LPN avg loss - NE avg loss: 1.7841943656655264e-15

Scale factor: 2
NE avg loss: 0.0021196380817564203
LPN avg loss: 0.03647896707919426
LPN avg loss - NE avg loss: 0.03435932899743784

Scale factor: 3
NE avg loss: 0.030496749129379167
LPN avg loss: 0.29332542669004763
LPN avg loss - NE avg loss: 0.26282867756066847

Scale factor: 4
NE avg loss: 0.1220150892091915
LPN avg loss: 0.9179058705996722
LPN avg loss - NE avg loss: 0.7958907813904808

Scale factor: 5
NE avg loss: 0.2963725308692083
LPN avg loss: 1.984061611548066
LPN avg loss - NE avg loss: 1.6876890806788578

